# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train
from torch.utils.data import random_split
from model.multimodal_har_model import MultiModalHARModel

2025-10-22 13:42:43,953 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-10-22 13:42:43,954 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


## Initializing Training

**Creating Dataloaders**

In [2]:
TRAIN_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "train"
)
TEST_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "test"
)

train_video_data_loader = VideoDataLoader(
    path=TRAIN_DIR,
)
test_video_data_loader = VideoDataLoader(
    path=TEST_DIR,
)

train_dataset = VideoDataset(
    video_data_loader=train_video_data_loader,
    normalization_type="across_frames",
)
test_dataset = VideoDataset(
    video_data_loader=test_video_data_loader,
    normalization_type="across_frames",
)

len(train_dataset)
for _ in train_dataset:
    pass
len(train_dataset.labels_map)

len(test_dataset)
for _ in test_dataset:
    pass
len(test_dataset.labels_map)

2025-10-22 13:42:45,917 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-22 13:42:46,144 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-22 13:42:46,452 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-22 13:42:46,773 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-22 13:42:47,180 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-22 13:42:47,395 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-22 13:42:47,621 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-22 13:42:48,196 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-22 13:42:48,676 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-22 13:42:48,908 - INFO - [VideoDataLoader] Loding action videos for action: a12
2025-10-22 13:42:53,692 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-22 13:42:53,756 - INFO -

10

**Splitting Train and Test Datasets**

In [3]:
# num_total = len(train_dataset)
# num_train = int(0.8 * num_total)
# num_test = num_total - num_train
# train_dataset, test_dataset = random_split(train_dataset, [num_train, num_test])

**Creating Model**

In [4]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(train_dataset.labels_map), 
    dropout=0.5,
)

## Training Model

In [5]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=25,
)

2025-10-22 13:42:55,262 - INFO - Starting training loop...
Epoch 1/25: 100%|██████████| 1176/1176 [06:55<00:00,  2.83it/s]
2025-10-22 13:49:51,176 - INFO - Epoch 1/25, Loss: 1.8997
Epoch 2/25: 100%|██████████| 1176/1176 [06:37<00:00,  2.96it/s]
2025-10-22 13:56:28,396 - INFO - Epoch 2/25, Loss: 1.4471
Epoch 3/25: 100%|██████████| 1176/1176 [06:38<00:00,  2.95it/s]
2025-10-22 14:03:07,197 - INFO - Epoch 3/25, Loss: 1.2742
Epoch 4/25: 100%|██████████| 1176/1176 [06:37<00:00,  2.96it/s]
2025-10-22 14:09:44,244 - INFO - Epoch 4/25, Loss: 1.1266
Epoch 5/25: 100%|██████████| 1176/1176 [33:07<00:00,  1.69s/it]   
2025-10-22 14:42:51,606 - INFO - Epoch 5/25, Loss: 1.0480
Epoch 6/25: 100%|██████████| 1176/1176 [06:38<00:00,  2.95it/s]
2025-10-22 14:49:30,002 - INFO - Epoch 6/25, Loss: 0.9766
Epoch 7/25: 100%|██████████| 1176/1176 [06:39<00:00,  2.94it/s]
2025-10-22 14:56:09,587 - INFO - Epoch 7/25, Loss: 0.9123
Epoch 8/25: 100%|██████████| 1176/1176 [07:04<00:00,  2.77it/s]
2025-10-22 15:03:14,

**Saving Model**

In [6]:
har_model.save(
    training_history=train_history
)

2025-10-22 17:14:29,583 - INFO - Saving model to /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v0.0.0_nw_ucla_2025-10-22 17:14:29.583586.pht...
2025-10-22 17:14:29,861 - INFO - Model saved successfully.


In [7]:
import torch
from datetime import datetime

model_settings = config.model_settings
save_dir = "~/masters-models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(
    save_dir,
    f"har_model_{model_settings.model_version}_{model_settings.dataset_prefix}_100perc_{datetime.now()}.pht"
)
torch.save({
"model_state_dict": har_model.state_dict(),
"training_history": train_history,
"model_config": har_model._model_config,
}, save_path)


## Running tests

**Accuracy**

In [8]:
def evaluate(model, dataset, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            output = model(graphs_objects, graphs_joints)

            # Compute prediction
            if output.dim() == 1:
                predicted = torch.argmax(output).unsqueeze(0)
            else:
                _, predicted = torch.max(output, dim=1)

            correct += (predicted == label).sum().item()
            total += 1

    accuracy = 100 * correct / total
    return accuracy


In [9]:
accuracy_evaluation = evaluate(har_model, test_dataset, "mps")
print(f"Test Accuracy: {accuracy_evaluation:.2f}%")

Test Accuracy: 71.24%
